# Data Import/Export

Working with data is a fundamental aspect of data science in Python,
with data import and export being crucial skills. Throughout, we will
use the 311 service request data for illustrations, downloaded from
the NYC Open Data as a `csv` file.

## Using the `Pandas` Package

The pandas library simplifies data manipulation and analysis. It's
especially handy for dealing with CSV files.


In [ ]:
import pandas as pd

# Define the file name
csvnm = "data/rodent_2022-2023.csv"

# Specify the strings that indicate missing values
# Q: How would you know these?
na_values = [
    "",
    "0 Unspecified",
    "N/A",
    "na",
    "na na",
    "Unspecified",
    "UNKNOWN",
]

def custom_date_parser(x):
    return pd.to_datetime(x, format="%m/%d/%Y %I:%M:%S %p", errors='coerce')

# Read the CSV file
df = pd.read_csv(
    csvnm,
    na_values = na_values,
    parse_dates = ['Created Date', 'Closed Date'], 
    date_parser = custom_date_parser,
    dtype = {'Latitude': 'float32', 'Longitude': 'float32'},
)

# Strip leading and trailing whitespace from the column names
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(' ', '_', regex = False).str.lower()

# Drop the 'Location' since it is redundant
# df.drop(columns=['Location'], inplace=True)

The `pandas` package also provides some utility functions for quick
summaries about the data frame.

In [ ]:
df.shape
df.describe()
df.isnull().sum()

What are the unique values of `descriptor`?

In [ ]:
df.descriptor.unique()

## Filling Missing Values

If geocodes are available but zip code is missing, we can use
reverse geocoding to fill the zip code. This process involves
querying a geocoding service with latitude and longitude to
get the corresponding address details, including the ZIP code.
This can be done with package `geopy`, which needs to be
installed first: `pip install geopy`.


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Function for reverse geocoding
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        address = location.raw.get('address', {})
        zip_code = address.get('postcode')
        return zip_code
    except (GeocoderTimedOut, GeocoderServiceError):
        # Handle errors or timeouts
        return None

# Apply reverse geocoding to fill missing ZIP codes
for index, row in df.iterrows():
    if pd.isnull(row['incident_zip']) and pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        df.at[index, 'incident_zip'] = reverse_geocode(row['latitude'], row['longitude'])

# Note: This can be slow for large datasets due to API rate
# limits and network latency

## Using Appache `Arrow` Library

To read and export data efficiently, leveraging the Apache `Arrow`
library can significantly improve performance and storage efficiency,
especially with large datasets. The IPC (Inter-Process Communication)
file format in the context of Apache Arrow is a key component for
efficiently sharing data between different processes, potentially
written in different programming languages. Arrow's IPC mechanism is
designed around two main file formats:

+ Stream Format: For sending an arbitrary length sequence of Arrow
record batches (tables). The stream format is useful for real-time
data exchange where the size of the data is not known upfront and can
grow indefinitely.
+ File (or Feather) Format: Optimized for storage and memory-mapped
access, allowing for fast random access to different sections of the
data. This format is ideal for scenarios where the entire dataset is
available upfront and can be stored in a file system for repeated
reads and writes.


Apache Arrow provides a columnar
memory format for flat and hierarchical data, optimized for efficient
data analytics. It can be used in Python through the `pyarrow`
package. Here's how you can use Arrow to read, manipulate, and export
data, including a demonstration of storage savings.


First, ensure you have pyarrow installed on your computer (and
preferrably, in your current virtual environment):
```
pip install pyarrow
```


Feather is a fast, lightweight, and easy-to-use binary file format for
storing data frames, optimized for speed and efficiency, particularly
for IPC and data sharing between Python and R.

In [ ]:
df.to_feather('data/rodent_2022-2023.feather')

Read the feather file back in:

In [ ]:
dff = pd.read_feather("data/rodent_2022-2023.feather")
dff.shape

Benefits of Using Feather:

+ Efficiency: Feather is designed to support fast reading and writing
of data frames, making it ideal for analytical workflows that need to
exchange large datasets between Python and R.
+ Compatibility: Maintains data type integrity across Python and R,
ensuring that numbers, strings, and dates/times are correctly handled
and preserved.
+ Simplicity: The API for reading and writing Feather files is
straightforward, making it accessible to users with varying levels of
programming expertise.

By using Feather format for data storage, you leverage a modern
approach optimized for speed and compatibility, significantly
enhancing the performance of data-intensive applications.

## Accessing the Census Data with `uszipcode`

First, ensure the DataFrame (df) is ready for merging with census
data. Specifically, check that the `incident_zip` column is clean
and consistent.

In [ ]:
print(df['incident_zip'].isnull().sum())
# Standardize to 5-digit codes, if necessary
df['incident_zip'] = df['incident_zip'].astype(str).str.zfill(5) 

We can use the `uszipcode` package to get basic demographic data
for each zip code. For more detailed or specific census data, 
using the `CensusData` package or direct API calls to the Census
Bureau's API.


The `uszipcode` package provides a range of information about
ZIP codes in the United States. When you query a ZIP code using
`uszipcode`, you can access various attributes related to
demographic data, housing, geographic location, and more. Here
are some of the key variables available at the ZIP code level:


**emographic Information**

+ `population`: The total population.
+ `population_density`: The population per square kilometer.
+ `housing_units`: The total number of housing units.
+ `occupied_housing_units`: The number of occupied housing units.
+ `median_home_value`: The median value of homes.
+ `median_household_income`: The median household income.
+ `age_distribution`: A breakdown of the population by age.

**Geographic Information**

+ `zipcode`: The ZIP code.
+ `zipcode_type`: The type of ZIP code (e.g., Standard, PO Box).
+ `major_city`: The major city associated with the ZIP code.
+ `post_office_city`: The city name recognized by the U.S. Postal Service.
+ `common_city_list`: A list of common city names for the ZIP code.
+ `county`: The county in which the ZIP code is located.
+ `state`: The state in which the ZIP code is located.
+ `lat`: The latitude of the approximate center of the ZIP code.
+ `lng`: The longitude of the approximate center of the ZIP code.
+ `timezone`: The timezone of the ZIP code.

**Economic and Housing Data**

+ `land_area_in_sqmi`: The land area in square miles.
+ `water_area_in_sqmi`: The water area in square miles.
+ `occupancy_rate`: The rate of occupancy for housing units.
+ `median_age`: The median age of the population.


Install the `uszipcode` package into the current virtual environment
by `pip install uszipcode`. 

Now let's work on the rodent sightings data.

We will first clean the incident_zip column to ensure it only
contains valid ZIP codes. Then, we will use a vectorized
approach to fetch the required data for each unique ZIP code
and merge this information back into the original `DataFrame`.


In [ ]:
# Remove rows where 'incident_zip' is missing or not a valid ZIP code format
valid_zip_df = df.dropna(subset=['incident_zip']).copy()
valid_zip_df['incident_zip'] = valid_zip_df['incident_zip'].astype(str).str.zfill(5)
unique_zips = valid_zip_df['incident_zip'].unique()

Since `uszipcode` doesn't inherently support vectorized operations
for multiple ZIP code queries, we'll optimize the process by
querying each unique ZIP code once, then merging the results
with the original `DataFrame`. This approach minimizes redundant
queries for ZIP codes that appear multiple times.


In [ ]:
from uszipcode import SearchEngine

# Initialize the SearchEngine
search = SearchEngine()

# Fetch median home value and median household income for each unique ZIP code
zip_data = []
zip_data = []
for zip_code in unique_zips:
    result = search.by_zipcode(zip_code)
    if result:  # Check if the result is not None
        zip_data.append({
            "incident_zip": zip_code,
            "median_home_value": result.median_home_value,
            "median_household_income": result.median_household_income
        })
    else:  # Handle the case where the result is None
        zip_data.append({
            "incident_zip": zip_code,
            "median_home_value": None,
            "median_household_income": None
        })

# Convert to DataFrame
zip_info_df = pd.DataFrame(zip_data)

# Merge this info back into the original DataFrame based on 'incident_zip'
merged_df = pd.merge(valid_zip_df, zip_info_df, how="left", on="incident_zip")

merged_df.columns